# Introduction

Data consists of features extacted from microscopic images of breast tissue. The images are classified as *malignant* or *benign* based on the tissue it cam from. The task is to see if we can train a model to predict if the tumor is malignant or benign from the given features. 

### Summary

Exploratory analysis of this data set showed that the features are encoded on ordinal scale.
Some of the features are highly skewed.
Some features have high collinearity.
The target classes are imbalanced. 
A tree based model was determined to be appropriate due to the skewness in the features as well as their collinearity. XGBoost with *gbtree* method was used to fit the data.
Adjusting for target class imbalance improved model performance significantly.

#### Result

Initial accuracy using *gbtree* booster of XGBoost was 95%.

#### Improvement

Balancing the classes using bootstrapping to simulate the data improved the prediction accuracy.

### Import the libraries

In [1]:
import pandas as pd
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import CategoricalColorMapper, LinearColorMapper
from bokeh.palettes import d3, Category10, Spectral10
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from math import pi
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 999

### Read the data

Read the data in pandas and display the data types of all the columns. Make sure the target labels are of the 'category' type.

In [2]:
AllData = pd.read_csv('../Data/breast-cancer-wisconsin.data', header = None)
AllData.columns = ['Sample code number','Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion','Single Epithelial Cell Size', 'Bare Nuclei','Bland Chromatin', 'Normal Nucleoli','Mitoses', 'Class']
display(AllData.head())
AllData['Class'] = AllData['Class'].astype('category')
display(AllData.dtypes)

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


Sample code number                int64
Clump Thickness                   int64
Uniformity of Cell Size           int64
Uniformity of Cell Shape          int64
Marginal Adhesion                 int64
Single Epithelial Cell Size       int64
Bare Nuclei                      object
Bland Chromatin                   int64
Normal Nucleoli                   int64
Mitoses                           int64
Class                          category
dtype: object

Looking at the data types above it is possible that *Bare Nuclei* column might have some weird values. A closer inspection is required.

In [3]:
temp = pd.Series(AllData["Bare Nuclei"], dtype="category")
display (temp.cat.categories)

Index(['1', '10', '2', '3', '4', '5', '6', '7', '8', '9', '?'], dtype='object')

Some of the values is a '?'.

In [4]:
display(AllData['Bare Nuclei'][AllData["Bare Nuclei"] == "?"])
print ("A total of {} values in the bare nuclei column are missing.".format(len(AllData['Bare Nuclei'][AllData["Bare Nuclei"] == "?"])))

23     ?
40     ?
139    ?
145    ?
158    ?
164    ?
235    ?
249    ?
275    ?
292    ?
294    ?
297    ?
315    ?
321    ?
411    ?
617    ?
Name: Bare Nuclei, dtype: object

A total of 16 values in the bare nuclei column are missing.


We could substitute these missing vlaues with the median value for this column. Since the number is small we can just delete these rows as well.

View some summary statistics for all the columns to make sure the distributions are not very skewed.

In [5]:
display(AllData.describe())

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bland Chromatin,Normal Nucleoli,Mitoses
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


From the above table it is clear that all the features range from 1.0 to 10.0. However, they differ widely in their distribution. Some features like *Bland Chromatin* have a good distribution around the mean. But some other features like *Mitoses* is extremely skewed.

It is also not clear if the values for these feature are discrete or ordinal. Perhapse, it is better to use tree based models to avoid these issues of skewness and ordinal values.

##### Are the classes balanced?

In [6]:
AllData['Class'].hist()

It is clear fromt the above plot that the target classes are not balanced. The number of *malignant* tumors are about half that of the *benign* tumors in the dataset. We can use bootstrapping to simulate some *malignant* data to balance the classes.

##### Are the features correlated?

In [7]:
display (AllData.corr())
print (AllData.corr().columns)

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bland Chromatin,Normal Nucleoli,Mitoses
Sample code number,1.000000,-0.055308,-0.041603,-0.041576,-0.064878,-0.045528,-0.060051,-0.052072,-0.034901
Clump Thickness,-0.055308,1.000000,0.644913,0.654589,0.486356,0.521816,0.558428,0.535835,0.350034
Uniformity of Cell Size,-0.041603,0.644913,1.000000,0.906882,0.705582,0.751799,0.755721,0.722865,0.458693
Uniformity of Cell Shape,-0.041576,0.654589,0.906882,1.000000,0.683079,0.719668,0.735948,0.719446,0.438911
Marginal Adhesion,-0.064878,0.486356,0.705582,0.683079,1.000000,0.599599,0.666715,0.603352,0.417633
Single Epithelial Cell Size,-0.045528,0.521816,0.751799,0.719668,0.599599,1.000000,0.616102,0.628881,0.479101
Bland Chromatin,-0.060051,0.558428,0.755721,0.735948,0.666715,0.616102,1.000000,0.665878,0.344169
Normal Nucleoli,-0.052072,0.535835,0.722865,0.719446,0.603352,0.628881,0.665878,1.000000,0.428336
Mitoses,-0.034901,0.350034,0.458693,0.438911,0.417633,0.479101,0.344169,0.428336,1.000000


Index(['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size',
       'Uniformity of Cell Shape', 'Marginal Adhesion',
       'Single Epithelial Cell Size', 'Bland Chromatin', 'Normal Nucleoli',
       'Mitoses'],
      dtype='object')


In [8]:
df = pd.DataFrame(AllData.corr().stack()).reset_index()
df["level_0"] = df["level_0"].astype(str)
df.columns = ["X", "Y", "Values"]
print (df.columns)

Index(['X', 'Y', 'Values'], dtype='object')


In [9]:
output_notebook()
#colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
my_colors = Spectral10
mapper = LinearColorMapper(palette=my_colors, low=df.Values.min(), high=df.Values.max())

p_heatmap = figure(title="Correlation Heat Map",
           x_range=list(AllData.corr().columns), y_range=list(reversed(AllData.corr().columns)),
           x_axis_location="above", plot_width=900, plot_height=900,
           toolbar_location='below')
p_heatmap.xaxis.major_label_orientation = pi / 3

p_heatmap.rect(x="X", y="Y", width=1, height=1,
       source=df,
       fill_color={'field': 'Values', 'transform': mapper},
       line_color=None)

show(p_heatmap)

Loading BokehJS ...

Looks like some of the features are highly correlated with each other. Particularly, *Uniformity of Cell Size* and *Uniformity of Cell Shape* are highly collinear. We can probably just use one of these features in the model.

Now we can visualize how the target might be associated with the features.

In [10]:
MyColors = np.repeat('blue', len(AllData))
MyColors[AllData['Class'] == 4] = 'red'
output_notebook()
p = figure(plot_width=400, plot_height=400,title="Malignancy by Clump Thinckness and Bland Chromatin")
#pallette = d3['Category10'][5]
#pallette = Spectral2
#color_map = CategoricalColorMapper(factors=AllData['Class'].unique(),palette=pallette)
# add a circle renderer with a size, color, and alpha
p.circle(x= AllData['Clump Thickness'],y=  AllData['Bland Chromatin'], size=20, alpha=0.5, color = MyColors)
p.xaxis.axis_label = "Clump Thickness"
p.yaxis.axis_label = "Bland Chromatin"

# show the results
show(p)

Loading BokehJS ...

The plot above shows that as both the clump thickness and bland chromatin increase, the chances of having malignant cancer also increase.

Make sure there are no missing values in the data.

In [11]:
AllCoumnNames = AllData.columns
[(print (CurrentColumn), print (AllData[CurrentColumn].unique())) for CurrentColumn in AllCoumnNames[1:]]
len(AllData['Bare Nuclei'])

Clump Thickness
[ 5  3  6  4  8  1  2  7 10  9]
Uniformity of Cell Size
[ 1  4  8 10  2  3  7  5  6  9]
Uniformity of Cell Shape
[ 1  4  8 10  2  3  5  6  7  9]
Marginal Adhesion
[ 1  5  3  8 10  4  6  2  9  7]
Single Epithelial Cell Size
[ 2  7  3  1  6  4  5  8 10  9]
Bare Nuclei
['1' '10' '2' '4' '3' '9' '7' '?' '5' '8' '6']
Bland Chromatin
[ 3  9  1  2  4  5  7  8  6 10]
Normal Nucleoli
[ 1  2  7  4  5  3 10  6  9  8]
Mitoses
[ 1  5  4  2  3  7 10  8  6]
Class
[2, 4]
Categories (2, int64): [2, 4]


699

Looks like 'bare nuclei' has some missing values.

So check how many rows have the missing values.

In [12]:
len(AllData['Bare Nuclei'][AllData['Bare Nuclei'] == '?'])

16

Only 16 of 699 rows have missing data. So just remove the rows that have missing data.

In [23]:
RowsToKeep = AllData['Bare Nuclei'] != '?'
AllDataMod = pd.DataFrame(AllData[RowsToKeep],copy=True)
AllDataMod['Bare Nuclei'] = AllDataMod['Bare Nuclei'].astype('int')
AllDataMod

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2
8,1033078,2,1,1,1,2,1,1,1,5,2
9,1033078,4,2,1,1,2,1,2,1,1,2


#### Validation

Perform some validation to tune the hyperparameters. Split the data into five subsets to do K-fold cross-validation. We can use this to choose which booster works better with the data.

First split the data into training and testing.

In [24]:
FeaturesToUse = ['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion','Single Epithelial Cell Size', 'Bare Nuclei','Bland Chromatin', 'Normal Nucleoli','Mitoses']
Feature_X = AllDataMod[FeaturesToUse]
Target_Y = AllDataMod['Class']

In [25]:
Split_size = 0.25
Training_X, Testing_X, Training_Y, Testing_Y = train_test_split(Feature_X, Target_Y, test_size=Split_size, stratify = AllDataMod['Class'])



In [26]:
print (Training_X.shape)
print (Testing_X.shape)
print (Training_Y.shape)
print (Testing_Y.shape)

(512, 9)
(171, 9)
(512,)
(171,)


In [27]:
AccuracyArray = pd.Series()
ModelToUse = ['gbtree']
MaxDepth = [1,2,4,6,8,10,100]
eta_list = [0.0001,0.001,0.01,0.1,0.2,0.5,1]
kf = KFold(n_splits=5)
kf.get_n_splits(Training_X)
print (kf)
def train_model (Training_X, Training_Y, Current_eta):
    for train_index, test_index in kf.split(Training_X):
        AccuracyArray = pd.Series()
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = Training_X.iloc[train_index], Training_X.iloc[test_index]
        y_train, y_test = Training_Y.iloc[train_index], Training_Y.iloc[test_index]
        Model_2 = XGBClassifier(booster = 'gbtree', eta=Current_eta)
        Model_2.fit(X_train, y_train)
        TempPredictions = Model_2.predict(X_test)
        TempAccuracy = accuracy_score(y_test, TempPredictions)
        TempAccuracy = round ((TempAccuracy*100),2)
        AccuracyArray = AccuracyArray.append(pd.Series(TempAccuracy))
    print ('Average validation accuracy with eta {} = {}'.format (Current_eta,round(AccuracyArray.mean(), 2)))
for Current_eta in eta_list:
    train_model(Training_X, Training_Y, Current_eta)
#print ('Average validation accuracy with {}= {}'.format (ModelToUse[1],round(AccuracyArray.mean(), 2)))

KFold(n_splits=5, random_state=None, shuffle=False)
Average validation accuracy with eta 0.0001 = 96.08
Average validation accuracy with eta 0.001 = 96.08
Average validation accuracy with eta 0.01 = 96.08
Average validation accuracy with eta 0.1 = 96.08
Average validation accuracy with eta 0.2 = 96.08
Average validation accuracy with eta 0.5 = 96.08
Average validation accuracy with eta 1 = 96.08


It looks like the model performs equally well for a range of *learning rate* parameters.

#### Train model

Train the model using the selected hyperparameters.

In [28]:
Model = XGBClassifier(booster='gbtree')
Model.fit(Training_X, Training_Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [29]:
Predicted_Y = Model.predict(Testing_X)
#predictions = [round(value) for value in y_pred]
Predicted_Y

array([4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 4, 4, 2, 2, 2, 4,
       2, 2, 4, 2, 2, 4, 4, 4, 4, 2, 2, 4, 2, 2, 4, 4, 2, 4, 2, 4, 2, 2,
       4, 2, 4, 2, 4, 4, 4, 2, 4, 2, 4, 2, 2, 2, 4, 2, 4, 4, 2, 2, 4, 2,
       2, 4, 4, 2, 4, 4, 2, 4, 2, 2, 4, 2, 4, 4, 4, 2, 2, 4, 2, 4, 2, 4,
       2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2,
       2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2,
       2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 4, 4, 4, 4, 2, 4, 4,
       2, 2, 2, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2])

In [30]:
Accuracy = accuracy_score(Testing_Y, Predicted_Y)
print ('Accuracy: {}%'.format(round((Accuracy*100), 2)))

Accuracy: 96.49%


In [31]:
confusion_matrix(Testing_Y, Predicted_Y)

array([[107,   4],
       [  2,  58]])

In [32]:
tn, fp, fn, tp = confusion_matrix(Testing_Y, Predicted_Y).ravel()
print ("True Negatives = {}, True Positives = {}, False Nagatives = {}, False Positives = {}".format(tn,tp,fn,fp))

True Negatives = 107, True Positives = 58, False Nagatives = 2, False Positives = 4


Now let's see if we can get better results with simulated data.

Now let's try upsampling *malignant* data to see if we can see some improvements.

In [40]:
MyClasses = pd.Series("Benign")
MyClasses = MyClasses.repeat(len(AllDataMod))
MyClasses[pd.Series(AllDataMod["Class"] == 4).values] = "Malignant"
AllDataMod["Class Text"] = MyClasses.values
AllDataMod["Class Text"] = AllDataMod["Class Text"].astype('category')
AllDataMod.head()

FeaturesToUse = ['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion','Single Epithelial Cell Size', 'Bare Nuclei','Bland Chromatin', 'Normal Nucleoli','Mitoses']
Feature_X_Sim = AllDataMod[FeaturesToUse]
Target_Y_Sim = AllDataMod["Class Text"]

Split_size = 0.25
Training_X_Sim, Testing_X_Sim, Training_Y_Sim, Testing_Y_Sim = train_test_split(Feature_X_Sim, Target_Y_Sim, test_size=Split_size, stratify = AllDataMod['Class Text'])
###########
Training_X_Sim = Training_X_Sim.reset_index()
Training_X_Sim = Training_X_Sim.iloc[:,1:]
Training_Y_Sim = Training_Y_Sim.reset_index()
Training_Y_Sim = Training_Y_Sim.iloc[:,1:]
Training_X_Sim_Up = Training_X_Sim[(Training_Y_Sim == "Benign")["Class Text"]]
MalignantIndex = pd.Series(Training_Y_Sim.index.values[(Training_Y_Sim == "Malignant")["Class Text"]])
MalignantIndex = MalignantIndex.sample(n=len(Training_X_Sim_Up), replace=True)
Training_X_Sim_Up = Training_X_Sim_Up.append(Training_X_Sim.iloc[MalignantIndex.values,:])
Training_X_Sim_Up = Training_X_Sim_Up.sort_index()
#display (Training_X_Sim_Up)
#### Now Testing data ####
Training_Y_Sim_Up = Training_Y_Sim[(Training_Y_Sim == "Benign")["Class Text"]]
Training_Y_Sim_Up = Training_Y_Sim_Up.append(Training_Y_Sim.iloc[MalignantIndex.values,:])
Training_Y_Sim_Up = Training_Y_Sim_Up.sort_index()
#display(Training_Y_Sim_Up)




In [41]:
Model_Sim = XGBClassifier(booster='gbtree')
Model_Sim.fit(Training_X_Sim_Up, Training_Y_Sim_Up)
######
Predicted_Y_Sim = Model_Sim.predict(Testing_X_Sim)
Predicted_Y_Sim
Accuracy_Sim = accuracy_score(Testing_Y_Sim, Predicted_Y_Sim)
print ('Accuracy: {}%'.format(round((Accuracy_Sim*100), 2)))
confusion_matrix(Testing_Y_Sim, Predicted_Y_Sim)

Accuracy: 98.25%


array([[108,   3],
       [  0,  60]])

In [42]:
tn, fp, fn, tp = confusion_matrix(Testing_Y_Sim, Predicted_Y_Sim).ravel()
print ("True Negatives = {}, True Positives = {}, False Nagatives = {}, False Positives = {}".format(tn,tp,fn,fp))

True Negatives = 108, True Positives = 60, False Nagatives = 0, False Positives = 3


Our accuracy jumped up after balancing the classes. It looks like we can predict with pretty high confidence whether a tissue is *malignant* or *benign* based on the available data.